In [16]:
import sympy as sp

In [17]:
# State variables
x_SoC, T_c, T_h = sp.symbols('x_SoC, T_c, T_h')

# Input variables
I_HP, x_FAN = sp.symbols('I_HP, x_FAN')

# Battery parameters
Q_max, R_in, P_rest = sp.symbols('Q_max, R_in, P_rest')
a3, a2, a1, a0 = sp.symbols('a3, a2, a1, a0')
U_oc = a3 * x_SoC**3 + a2 * x_SoC**2 + a1 * x_SoC + a0

# Fan parameters
U_FAN, I_FAN = sp.symbols('U_FAN, I_FAN')

# LED parameters
I_LED, x_LED, T_LED = sp.symbols('I_LED, x_LED, T_LED')

# Peltier module parameters
S_M, R_M, K_M, x_HP = sp.symbols('S_M, R_M, K_M, x_HP')

# Thermal parameters
m_h, c_p, m_c, R_eq, R3_lambda, T_amb = sp.symbols('m_h, c_p, m_c, R_eq, R3_lambda, T_amb') # TODO change R_eq to its expression

# Define the three nonlinear differential equations
dTh_dt = (1 / (m_h * c_p)) * (S_M * I_HP * T_c - (R_in + 0.5 * R_M) * I_HP**2 - K_M * T_h + K_M * T_c + U_oc * I_HP - R_in * (I_FAN + I_LED) * I_HP + T_amb / R_eq - T_h / R_eq)
dTc_dt = (1 / (m_c * c_p)) * (T_LED / R3_lambda - (1 / R3_lambda + K_M) * T_c - S_M * I_HP * T_c + 0.5 * R_M * I_HP**2 + K_M * T_h)
dxSoC_dt = (U_oc + R_in * x_HP * I_HP + R_in * I_LED * x_LED - sp.sqrt(U_oc**2 - 2 * R_in * I_LED * x_LED * U_oc - 2 * R_in * x_HP * U_oc * I_HP + (R_in * x_HP)**2 * I_HP**2 + 2 * R_in * I_LED * x_LED * R_in * x_HP * I_HP - 4 * R_in * I_FAN * U_FAN * x_FAN - 4 * R_in * P_rest + (R_in * I_LED * x_LED)**2)) / (-2 * Q_max * R_in)

# Define state and input vectors
x = sp.Matrix([x_SoC, T_c, T_h])
u = sp.Matrix([I_HP, x_FAN])

# Define the system of differential equations as a matrix
f = sp.Matrix([dxSoC_dt, dTc_dt, dTh_dt])

# Compute Jacobians for A and B matrices
A = f.jacobian(x)
B = f.jacobian(u)

In [18]:
# # Simplified version
# # Define state and input vectors
# x = sp.Matrix([T_c, T_h])
# u = sp.Matrix([I_HP])

# # Define the system of differential equations as a matrix
# f = sp.Matrix([dTc_dt, dTh_dt])

# # Compute Jacobians for A and B matrices
# A = f.jacobian(x)
# B = f.jacobian(u)

In [19]:
A_latex = sp.latex(A)
B_latex = sp.latex(B)

# Print LaTeX code
print("LaTeX representation of Matrix A:")
print(A_latex)

print("\nLaTeX representation of Matrix B:")
print(B_latex)


LaTeX representation of Matrix A:
\left[\begin{matrix}- \frac{a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2} - \frac{- I_{HP} R_{in} x_{HP} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right) - I_{LED} R_{in} x_{LED} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right) + \frac{\left(2 a_{1} + 4 a_{2} x_{SoC} + 6 a_{3} x_{SoC}^{2}\right) \left(a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right)}{2}}{\sqrt{- 4 I_{FAN} R_{in} U_{FAN} x_{FAN} + I_{HP}^{2} R_{in}^{2} x_{HP}^{2} + 2 I_{HP} I_{LED} R_{in}^{2} x_{HP} x_{LED} - 2 I_{HP} R_{in} x_{HP} \left(a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) + I_{LED}^{2} R_{in}^{2} x_{LED}^{2} - 2 I_{LED} R_{in} x_{LED} \left(a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) - 4 P_{rest} R_{in} + \left(a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right)^{2}}}}{2 Q_{max} R_{in}} & 0 & 0\\0 & \frac{- I_{HP} S_{M} - K_{M} - \frac{1}{R_{3 \lambda}}}{c_{p} m_{c}} & \fr

In [20]:
A

Matrix([
[-(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2 - (-I_HP*R_in*x_HP*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2) - I_LED*R_in*x_LED*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2) + (2*a1 + 4*a2*x_SoC + 6*a3*x_SoC**2)*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3)/2)/sqrt(-4*I_FAN*R_in*U_FAN*x_FAN + I_HP**2*R_in**2*x_HP**2 + 2*I_HP*I_LED*R_in**2*x_HP*x_LED - 2*I_HP*R_in*x_HP*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) + I_LED**2*R_in**2*x_LED**2 - 2*I_LED*R_in*x_LED*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) - 4*P_rest*R_in + (a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3)**2))/(2*Q_max*R_in),                                         0,                         0],
[                                                                                                                                                                                                                                                                                                                                                                           

In [21]:
B

Matrix([
[-(R_in*x_HP - (I_HP*R_in**2*x_HP**2 + I_LED*R_in**2*x_HP*x_LED - R_in*x_HP*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3))/sqrt(-4*I_FAN*R_in*U_FAN*x_FAN + I_HP**2*R_in**2*x_HP**2 + 2*I_HP*I_LED*R_in**2*x_HP*x_LED - 2*I_HP*R_in*x_HP*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) + I_LED**2*R_in**2*x_LED**2 - 2*I_LED*R_in*x_LED*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) - 4*P_rest*R_in + (a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3)**2))/(2*Q_max*R_in), -I_FAN*U_FAN/(Q_max*sqrt(-4*I_FAN*R_in*U_FAN*x_FAN + I_HP**2*R_in**2*x_HP**2 + 2*I_HP*I_LED*R_in**2*x_HP*x_LED - 2*I_HP*R_in*x_HP*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) + I_LED**2*R_in**2*x_LED**2 - 2*I_LED*R_in*x_LED*(a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) - 4*P_rest*R_in + (a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3)**2))],
[                                                                                                                                                                                                           

In [22]:
# # x_SoC_ss, T_c_ss, T_h_ss = sp.symbols('x_SoC_ss T_c_ss T_h_ss')
# # I_HP_ss, I_FAN_ss = sp.symbols('I_HP_ss I_LED_ss')
# T_c_ss, T_h_ss = sp.symbols('T_c_ss T_h_ss')
# I_HP_ss = sp.symbols('I_HP_ss')

# # x_ss = sp.Matrix([x_SoC_ss, T_c_ss, T_h_ss])
# x_ss = sp.Matrix([T_c_ss, T_h_ss])
# # u_ss = sp.Matrix([I_HP_ss, I_FAN_ss])
# u_ss = sp.Matrix([I_HP_ss])

# steady_state_linear_eq = A * sp.Matrix([x_ss]) #+ B * sp.Matrix([u_ss])

# # Solve linear steady-state
# linear_steady_state = sp.solve(steady_state_linear_eq, [x_ss], dict=True)
# print("\nLinear steady-state solutions:")
# sp.pprint(linear_steady_state)

In [23]:
# # Set derivatives to zero for steady-state
# steady_state_eqs = [eq for eq in f]

# # Solve steady-state equations
# steady_state = sp.solve(steady_state_eqs, [x_SoC, T_c, T_h], dict=True)
# print("Nonlinear steady-state solutions:")
# sp.pprint(steady_state)